In [1]:
import json
import logging
import re
import warnings
from datetime import datetime
from pathlib import Path
from pprint import pprint
from typing import Annotated, Any, Generator, Literal, Type, TypeVar

# Standard imports
import numpy as np
import numpy.typing as npt
import pandas as pd
import polars as pl

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [ ]:
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme({
    "white": "#FFFFFF",  # Bright white
    "info": "#00FF00",  # Bright green
    "warning": "#FFD700",  # Bright gold
    "error": "#FF1493",  # Deep pink
    "success": "#00FFFF",  # Cyan
    "highlight": "#FF4500",  # Orange-red
})
console = Console(theme=custom_theme)


def create_path(path: str | Path) -> None:
    """
    Create parent directories for the given path if they don't exist.

    Parameters
    ----------
    path : str | Path
        The file path for which to create parent directories.

    """
    Path(path).parent.mkdir(parents=True, exist_ok=True)


def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)

In [3]:
go_up_from_current_directory(go_up=1)

/Users/mac/Desktop/MyProjects/batch-process


In [ ]:
from sqlalchemy import delete, insert, select, update

from schemas import EmailSchema
from src.database.db_models import EmailLog, get_db_session, init_db

Connected to 'test' environment database.


In [5]:
init_db()

## [Docs](https://docs.sqlalchemy.org/en/20/orm/queryguide/select.html)

### [Insert](https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-bulk-insert-statements)

- Old API

```python
with get_db_session() as session:
    data_dict = input_data.to_data_model_dict()
    record = EmailLog(**data_dict)
    session.add(record)
    session.flush()
    output_data = {key: getattr(record, key) for key in record.output_fields()}
```

<br>

- New API

```py
with get_db_session() as session:
    data_dict = input_data.to_data_model_dict()
    session.execute(insert(EmailLog), [data_dict])
```

In [6]:
input_data: EmailSchema = EmailSchema(
    recipient="marketing@client.com",
    subject="Partnership Proposal",
    body="We would like to discuss a potential partnership opportunity.",
)
console.print(input_data)

EmailSchema(
    recipient='marketing@client.com',
    subject='Partnership Proposal',
    body='We would like to discuss a potential partnership opportunity.',
    status='processing',
    created_at=datetime.datetime(2025, 7, 14, 20, 33, 41, 929315),
    sent_at=None
)

In [ ]:
input_data.model_dump()

{'recipient': 'marketing@client.com',
 'subject': 'Partnership Proposal',
 'body': 'We would like to discuss a potential partnership opportunity.',
 'status': 'processing',
 'created_at': '2025-07-14T20:33:41.929315',
 'sent_at': None}

In [9]:
with get_db_session() as session:
    data_dict = input_data.model_dump()
    record = EmailLog(**data_dict)
    session.add(record)
    session.flush()
    output_data = {key: getattr(record, key) for key in record.output_fields()}


console.print(output_data)

{
    'id': 1,
    'recipient': 'marketing@client.com',
    'subject': 'Partnership Proposal',
    'status': 'processing',
    'sent_at': None,
    'created_at': datetime.datetime(2025, 7, 14, 20, 33, 41, 929315)
}

In [13]:
with get_db_session() as session:
    statement = session.query(EmailLog).where(EmailLog.created_at < datetime.now())
    record = session.execute(statement).scalar_one()
    output_data = {key: getattr(record, key) for key in record.output_fields()}


console.print(output_data)

{
    'id': 1,
    'recipient': 'marketing@client.com',
    'subject': 'Partnership Proposal',
    'status': 'processing',
    'sent_at': None,
    'created_at': datetime.datetime(2025, 7, 14, 20, 33, 41, 929315)
}

In [ ]:
input_data_2: EmailSchema = EmailSchema(
    recipient="emeka2@example.com",
    subject="test!!!",
    body="this is an example body",
    status="processing",
)
input_data_3: EmailSchema = EmailSchema(
    recipient="john.doe@example.com",
    subject="Meeting Reminder",
    body="Hi John, just a reminder about our meeting tomorrow at 10 AM.",
    status="processing",
)
input_data_4: EmailSchema = EmailSchema(
    recipient="info@company.org",
    subject="New Product Launch",
    body="Dear valued customer, check out our exciting new product!",
    status="sent",
    created_at=datetime(2025, 7, 10, 9, 0, 0),
    sent_at="2025-07-10T09:05:00",
)
console.print((input_data_2, input_data_3, input_data_4))

### [Bulk Insert](https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-bulk-insert-statements)

- Old API

```py
with get_db_session() as session:
    data_list: list[dict[str, Any]] = [_data.to_data_model_dict() for _data in (input_data_2, input_data_3, input_data_4)]
    session.bulk_insert_mappings(EmailLog, data_list)
```

<br>

- New API

```py
with get_db_session() as session:
    data_list: list[dict[str, Any]] = [
        _data.to_data_model_dict()
        for _data in (input_data_2, input_data_3, input_data_4)
    ]
    session.execute(insert(EmailLog), data_list)
```

In [ ]:
with get_db_session() as session:
    data_list: list[dict[str, Any]] = [_data.model_dump() for _data in (input_data_2, input_data_3, input_data_4)]
    session.execute(insert(EmailLog), data_list)

### Select

In [ ]:
# Select a single record
with get_db_session() as session:
    statement = select(EmailLog).where(EmailLog.id == 1, EmailLog.status == "pending")
    record = session.execute(statement).scalar_one()
    output_data = {key: getattr(record, key) for key in record.output_fields()}


console.print(output_data)

In [ ]:
# Select all records
with get_db_session() as session:
    statement = select(EmailLog)
    record = session.execute(statement).scalars()

    output_data = [{key: getattr(row, key) for key in row.output_fields()} for row in record]

console.print(output_data)

### [Update](https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-update-and-delete-with-custom-where-criteria)

In [ ]:
with get_db_session() as session:
    statement = (
        update(EmailLog)
        .where(EmailLog.id == 1)
        .values(status="sent", sent_at=datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    )
    # It closes the session and returns None
    session.execute(statement)

# Verify that the record was updated
with get_db_session() as session:
    statement = select(EmailLog)
    record = session.execute(statement).scalars()

    output_data = [{key: getattr(row, key) for key in row.output_fields()} for row in record]

console.print(output_data)

### [Delete](https://docs.sqlalchemy.org/en/20/orm/queryguide/dml.html#orm-update-and-delete-with-custom-where-criteria)

In [ ]:
with get_db_session() as session:
    statement = delete(EmailLog).where(EmailLog.id == 2)
    # It closes the session and returns None
    session.execute(statement)

# Verify that the record was updated
with get_db_session() as session:
    statement = select(EmailLog)
    record = session.execute(statement).scalars()

    output_data = [{key: getattr(row, key) for key in row.output_fields()} for row in record]

console.print(output_data)

In [ ]:
from config import app_config

In [ ]:
beat_dict: dict[str, dict[str, Any]] = dict(app_config.celery_config.beat_config.beat_schedule.model_dump().items())

# Add the health_check
beat_dict["health_check"] = app_config.celery_config.beat_config.health_check.model_dump()


console.print(beat_dict)

In [ ]:
app_config.celery_config.beat_config.beat_schedule.model_dump().items()

In [ ]:
class TaskLog(Base):
    __tablename__ = "task_logs"

    id = Column(String, primary_key=True)
    task_id = Column(String)
    task_name = Column(String)
    status = Column(String)
    created_at = Column(DateTime)
    args = Column(Text)
    kwargs = Column(Text)
    result = Column(Text)
    error = Column(Text)


class DatabaseLoggingMixin:
    """Mixin class for database logging functionality"""

    def _save_log(self, task_id, status, **extra_data):
        session = Session()
        try:
            # Check if log entry already exists (for retries)
            existing_log = session.query(TaskLog).filter_by(id=task_id).first()

            if existing_log:
                # Update existing log
                existing_log.status = status
                existing_log.timestamp = datetime.now()
                for key, value in extra_data.items():
                    setattr(existing_log, key, value)
            else:
                # Create new log entry
                log_entry = TaskLog(
                    id=task_id, task_name=self.name, status=status, timestamp=datetime.now(), **extra_data
                )
                session.add(log_entry)

            session.commit()
        except Exception as e:
            print(f"Failed to save task log: {e}")
            session.rollback()
        finally:
            session.close()

    def on_success(self, retval, task_id, args, kwargs):
        self._save_log(
            task_id, "SUCCESS", args=json.dumps(args), kwargs=json.dumps(kwargs), result=str(retval)[:1000]
        )  # Truncate long results

        # Call parent's on_success if it exists
        if hasattr(super(), "on_success"):
            super().on_success(retval, task_id, args, kwargs)

    def on_failure(self, exc, task_id, args, kwargs, einfo):
        self._save_log(
            task_id, "FAILURE", args=json.dumps(args), kwargs=json.dumps(kwargs), error=str(exc)[:1000]
        )  # Truncate long errors

        # Call parent's on_failure if it exists
        if hasattr(super(), "on_failure"):
            super().on_failure(exc, task_id, args, kwargs, einfo)

    def on_retry(self, exc, task_id, args, kwargs, einfo):
        self._save_log(
            task_id,
            "RETRY",
            args=json.dumps(args),
            kwargs=json.dumps(kwargs),
            error=str(exc)[:1000],
            retry_count=self.request.retries,
        )

        # Call parent's on_retry if it exists
        if hasattr(super(), "on_retry"):
            super().on_retry(exc, task_id, args, kwargs, einfo)


class EmailTask(Task):
    autoretry_for = (Exception,)
    throws = (Exception,)  # Log full traceback on retry
    default_retry_delay = 30  # 30 seconds
    max_retries = 5
    retry_backoff = True  # exponential backoff
    retry_backoff_max = 300  # 10 minutes


# Combined class
class EmailTaskWithLogging(DatabaseLoggingMixin, EmailTask):
    """Email task with database logging capabilities"""

    pass

In [70]:
import json
from typing import Any
from pydantic import BaseModel, field_serializer
from datetime import datetime

class MyModel(BaseModel):
    name: str
    age: int
    role: str
    salary: float = 0.0
    others: Any | None = None

    @field_serializer("others")
    def serialize(self, value: Any) -> str:
        if isinstance(value, datetime):
            return value.isoformat()
        return json.dumps(value)


def my_func(name: str, **kwargs) -> MyModel:
    my_dict = {"name": name, **kwargs}
    return MyModel(**my_dict)


result = my_func(
    "Neidu",
    age=30,
    role="AI Engineer",
    friend="None",
    others=["Hi"],
    # others=datetime.now(),

)


In [71]:
print(result.model_dump())

json.loads(result.model_dump()["others"])

{'name': 'Neidu', 'age': 30, 'role': 'AI Engineer', 'salary': 0.0, 'others': '["Hi"]'}


['Hi']

In [ ]:
class CustomTask(Task):
    autoretry_for = (Exception,)
    throws = (Exception,)  # Log full traceback on retry
    default_retry_delay = 30  # 30 seconds
    max_retries = 5
    retry_backoff = True  # exponential backoff
    retry_backoff_max = 300  # 10 minutes

class CeleryTasksLog(Base):
    """Data model for storing Celery task meta."""

    __tablename__: str = "celery_tasks_log"
    id: Mapped[int] = mapped_column(primary_key=True)
    task_id: Mapped[str] = mapped_column("taskId", String(255), unique=True, index=True)
    task_name: Mapped[str] = mapped_column("taskName", String(255), index=True)
    status: Mapped[str] = mapped_column(String(50), default="PENDING")
    created_at: Mapped[str] = mapped_column("createdAt", DateTime(timezone=True), nullable=True)
    updated_at: Mapped[str] = mapped_column("updatedAt", DateTime(timezone=True), nullable=True)
    # args: MappedColumn[Any] = mapped_column(LargeBinary)
    kwargs: MappedColumn[Any] = mapped_column(LargeBinary)
    result: MappedColumn[Any] = mapped_column(LargeBinary)
    error: Mapped[str] = mapped_column(Text)

    def __repr__(self) -> str:
        """
        Returns a string representation of the email log.

        Returns
        -------
        str
        """
        return f"{self.__class__.__name__}(task_id={self.task_id!r}, created_at={self.created_at!r} status={self.status!r})"


# ===== Custom Mixins =====
class DatabaseLoggingMixin:
    def _save_log(self, task_id: str, status: str, **extra_data: dict[str, Any]) -> None:
        try:
            with get_db_session() as session:
                statement = session.query(CeleryTasksLog).where(CeleryTasksLog.task_id == task_id)
                existing_log = session.execute(statement).scalar_one()
                if existing_log:
                    # Update log
                    existing_log.status = status
                    existing_log.updated_at = datetime.now()
                    for key, value in extra_data.items():
                        setattr(existing_log, key, value)
                else:
                    # Create new log
                    _data = {
                        "task_id": task_id,
                        "status": status,
                        "updated_at": datetime.now(),
                    } | extra_data
                    data = CeleryTasksLogSchema(**_data).model_dump()  # type: ignore
                    new_log = CeleryTasksLog(**data)
                    session.add(new_log)
                    session.flush()
        except Exception as e:
            logger.error(f"Failed to save log: {e}")

    def on_success(self, retval: Any, task_id: str, args: Any, kwargs: Any) -> None:
        """Saves the task result and keyword arguments to the database on success.
        It must match the signature of the parent's on_success method.
        """
        retval = str(retval)[:1_000]  # Truncate long results
        self._save_log(task_id, status="SUCCESS", result=retval, kwargs=kwargs)
        # Call paret's on_success method if it exists
        if hasattr(super(), "on_success"):
            super().on_success(task_id, retval, kwargs)

    def on_failure(self, task_id: str, exc: Any, einfo: Any, kwargs: Any) -> None:
        einfo = str(einfo)[:1_000]  # Truncate long results
        self._save_log(task_id, status="FAILURE", error=einfo, kwargs=kwargs)
        # Call paret's on_failure method if it exists
        if hasattr(super(), "on_failure"):
            super().on_failure(task_id, exc, einfo, kwargs)

    def on_retry(self, task_id: str, exc: Any, einfo: Any, kwargs: Any) -> None:
        einfo = str(einfo)[:1_000]  # Truncate long results
        self._save_log(task_id, status="RETRY", error=einfo, kwargs=kwargs)
        # Call paret's on_retry method if it exists
        if hasattr(super(), "on_retry"):
            super().on_retry(task_id, exc, einfo, kwargs)


class BaseTask(DatabaseLoggingMixin, CustomTask):
    """Base class for tasks with database logging capabilities.

    Adds on_success, on_failure, and on_retry methods that save the task log to the database.
    """

    pass

3

In [ ]:
2025-07-15 21:43:19 - logger - [INFO] -  [+] Email sent to user4@example.com
[2025-07-15 21:43:19,352: INFO/ForkPoolWorker-4]  [+] Email sent to user4@example.com
2025-07-15 21:43:19 - database_utilities - [ERROR] - Database operation failed: No row was found when one was required
[2025-07-15 21:43:19,360: ERROR/ForkPoolWorker-4] Database operation failed: No row was found when one was required
2025-07-15 21:43:19 - database_utilities - [ERROR] - Failed to save log: No row was found when one was required
[2025-07-15 21:43:19,361: ERROR/ForkPoolWorker-4] Failed to save log: No row was found when one was required
[2025-07-15 21:43:19,395: ERROR/MainProcess] Task handler raised error: TypeError('DatabaseLoggingMixin.on_failure() takes 5 positional arguments but 6 were given')
billiard.einfo.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 529, in trace_task
    task_on_success(retval, uuid, args, kwargs)
  File "/Users/mac/Desktop/MyProjects/batch-process/src/database/db_models.py", line 287, in on_success
    super().on_success(task_id, retval, kwargs)
TypeError: Task.on_success() missing 1 required positional argument: 'kwargs'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/billiard/pool.py", line 362, in workloop
    result = (True, prepare_result(fun(*args, **kwargs)))
                                   ^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 651, in fast_trace_task
    R, I, T, Rstr = tasks[task].__trace__(
                    ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 574, in trace_task
    I, _, _, _ = on_error(task_request, exc)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 381, in on_error
    R = I.handle_error_state(
        ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 175, in handle_error_state
    return {
           ^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 233, in handle_failure
    task.on_failure(exc, req.id, req.args, req.kwargs, einfo)
TypeError: DatabaseLoggingMixin.on_failure() takes 5 positional arguments but 6 were given
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/billiard/pool.py", line 362, in workloop
    result = (True, prepare_result(fun(*args, **kwargs)))
                                   ^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 651, in fast_trace_task
    R, I, T, Rstr = tasks[task].__trace__(
                    ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 574, in trace_task
    I, _, _, _ = on_error(task_request, exc)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 381, in on_error
    R = I.handle_error_state(
        ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 175, in handle_error_state
    return {
           ^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 233, in handle_failure
    task.on_failure(exc, req.id, req.args, req.kwargs, einfo)
TypeError: DatabaseLoggingMixin.on_failure() takes 5 positional arguments but 6 were given
[2025-07-15 21:43:19,404: INFO/ForkPoolWorker-4] Task celery.chord_unlock[8799a7c9-4517-4718-a3f3-6cbcbe61bcf7] retry: Retry in 1.0s
[2025-07-15 21:43:19,408: INFO/MainProcess] Task src.celery.tasks.data_processing.process_data_chunk[d207a0d7-db0b-409b-b8dc-c4aa4e07da47] received
2025-07-15 21:43:31 - data_processing - [INFO] - Processed chunk 0 with 15 items in 13.61s
2025-07-15 21:43:31 - data_processing - [INFO] - Processed chunk 1 with 15 items in 13.61s
2025-07-15 21:43:31 - data_processing - [INFO] - Processed chunk 2 with 15 items in 13.60s
[2025-07-15 21:43:31,059: INFO/ForkPoolWorker-3] Processed chunk 0 with 15 items in 13.61s
[2025-07-15 21:43:31,059: INFO/ForkPoolWorker-1] Processed chunk 1 with 15 items in 13.61s
[2025-07-15 21:43:31,059: INFO/ForkPoolWorker-2] Processed chunk 2 with 15 items in 13.60s
2025-07-15 21:43:31 - database_utilities - [ERROR] - Database operation failed: No row was found when one was required
[2025-07-15 21:43:31,071: ERROR/ForkPoolWorker-1] Database operation failed: No row was found when one was required
2025-07-15 21:43:31 - database_utilities - [ERROR] - Database operation failed: No row was found when one was required
2025-07-15 21:43:31 - database_utilities - [ERROR] - Database operation failed: No row was found when one was required
[2025-07-15 21:43:31,071: ERROR/ForkPoolWorker-2] Database operation failed: No row was found when one was required
[2025-07-15 21:43:31,071: ERROR/ForkPoolWorker-3] Database operation failed: No row was found when one was required
2025-07-15 21:43:31 - database_utilities - [ERROR] - Failed to save log: No row was found when one was required
2025-07-15 21:43:31 - database_utilities - [ERROR] - Failed to save log: No row was found when one was required
[2025-07-15 21:43:31,071: ERROR/ForkPoolWorker-1] Failed to save log: No row was found when one was required
2025-07-15 21:43:31 - database_utilities - [ERROR] - Failed to save log: No row was found when one was required
[2025-07-15 21:43:31,072: ERROR/ForkPoolWorker-2] Failed to save log: No row was found when one was required
[2025-07-15 21:43:31,072: ERROR/ForkPoolWorker-3] Failed to save log: No row was found when one was required
[2025-07-15 21:43:31,109: ERROR/MainProcess] Task handler raised error: TypeError('DatabaseLoggingMixin.on_failure() takes 5 positional arguments but 6 were given')
billiard.einfo.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 529, in trace_task
    task_on_success(retval, uuid, args, kwargs)
  File "/Users/mac/Desktop/MyProjects/batch-process/src/database/db_models.py", line 287, in on_success
    super().on_success(task_id, retval, kwargs)
TypeError: Task.on_success() missing 1 required positional argument: 'kwargs'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/billiard/pool.py", line 362, in workloop
    result = (True, prepare_result(fun(*args, **kwargs)))
                                   ^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 651, in fast_trace_task
    R, I, T, Rstr = tasks[task].__trace__(
                    ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 574, in trace_task
    I, _, _, _ = on_error(task_request, exc)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 381, in on_error
    R = I.handle_error_state(
        ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 175, in handle_error_state
    return {
           ^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 233, in handle_failure
    task.on_failure(exc, req.id, req.args, req.kwargs, einfo)
TypeError: DatabaseLoggingMixin.on_failure() takes 5 positional arguments but 6 were given
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/billiard/pool.py", line 362, in workloop
    result = (True, prepare_result(fun(*args, **kwargs)))
                                   ^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 651, in fast_trace_task
    R, I, T, Rstr = tasks[task].__trace__(
                    ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 574, in trace_task
    I, _, _, _ = on_error(task_request, exc)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 381, in on_error
    R = I.handle_error_state(
        ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 175, in handle_error_state
    return {
           ^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 233, in handle_failure
    task.on_failure(exc, req.id, req.args, req.kwargs, einfo)
TypeError: DatabaseLoggingMixin.on_failure() takes 5 positional arguments but 6 were given
[2025-07-15 21:43:31,131: ERROR/MainProcess] Task handler raised error: TypeError('DatabaseLoggingMixin.on_failure() takes 5 positional arguments but 6 were given')
billiard.einfo.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 529, in trace_task
    task_on_success(retval, uuid, args, kwargs)
  File "/Users/mac/Desktop/MyProjects/batch-process/src/database/db_models.py", line 287, in on_success
    super().on_success(task_id, retval, kwargs)
TypeError: Task.on_success() missing 1 required positional argument: 'kwargs'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/billiard/pool.py", line 362, in workloop
    result = (True, prepare_result(fun(*args, **kwargs)))
                                   ^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 651, in fast_trace_task
    R, I, T, Rstr = tasks[task].__trace__(
                    ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 574, in trace_task
    I, _, _, _ = on_error(task_request, exc)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 381, in on_error
    R = I.handle_error_state(
        ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 175, in handle_error_state
    return {
           ^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 233, in handle_failure
    task.on_failure(exc, req.id, req.args, req.kwargs, einfo)
TypeError: DatabaseLoggingMixin.on_failure() takes 5 positional arguments but 6 were given
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/billiard/pool.py", line 362, in workloop
    result = (True, prepare_result(fun(*args, **kwargs)))
                                   ^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 651, in fast_trace_task
    R, I, T, Rstr = tasks[task].__trace__(
                    ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 574, in trace_task
    I, _, _, _ = on_error(task_request, exc)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 381, in on_error
    R = I.handle_error_state(
        ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 175, in handle_error_state
    return {
           ^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 233, in handle_failure
    task.on_failure(exc, req.id, req.args, req.kwargs, einfo)
TypeError: DatabaseLoggingMixin.on_failure() takes 5 positional arguments but 6 were given
[2025-07-15 21:43:31,154: ERROR/MainProcess] Task handler raised error: TypeError('DatabaseLoggingMixin.on_failure() takes 5 positional arguments but 6 were given')
billiard.einfo.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 529, in trace_task
    task_on_success(retval, uuid, args, kwargs)
  File "/Users/mac/Desktop/MyProjects/batch-process/src/database/db_models.py", line 287, in on_success
    super().on_success(task_id, retval, kwargs)
TypeError: Task.on_success() missing 1 required positional argument: 'kwargs'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/billiard/pool.py", line 362, in workloop
    result = (True, prepare_result(fun(*args, **kwargs)))
                                   ^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 651, in fast_trace_task
    R, I, T, Rstr = tasks[task].__trace__(
                    ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 574, in trace_task
    I, _, _, _ = on_error(task_request, exc)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 381, in on_error
    R = I.handle_error_state(
        ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 175, in handle_error_state
    return {
           ^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 233, in handle_failure
    task.on_failure(exc, req.id, req.args, req.kwargs, einfo)
TypeError: DatabaseLoggingMixin.on_failure() takes 5 positional arguments but 6 were given
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/billiard/pool.py", line 362, in workloop
    result = (True, prepare_result(fun(*args, **kwargs)))
                                   ^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 651, in fast_trace_task
    R, I, T, Rstr = tasks[task].__trace__(
                    ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 574, in trace_task
    I, _, _, _ = on_error(task_request, exc)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 381, in on_error
    R = I.handle_error_state(
        ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 175, in handle_error_state
    return {
           ^
  File "/Users/mac/Desktop/MyProjects/batch-process/.venv/lib/python3.12/site-packages/celery/app/trace.py", line 233, in handle_failure
    task.on_failure(exc, req.id, req.args, req.kwargs, einfo)
TypeError: DatabaseLoggingMixin.on_failure() takes 5 positional arguments but 6 were given
[2025-07-15 21:43:31,155: INFO/MainProcess] Task src.celery.tasks.data_processing.process_data_chunk[40120b3e-b944-4ab8-97eb-215445f69296] received
[2025-07-15 21:43:31,156: INFO/MainProcess] Task src.celery.tasks.data